In [1]:
import json
import string
import random 
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer 
import tensorflow as tensorF
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\devlo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\devlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
f=open('ourdata.json')
data=json.load(f)

In [3]:
for intents in data:
  print(data[intents]['Responses'][1])

Result sent 
Dear Student,

as per your request your course has been dropped
You will get the timetable before the commencement of classes.
Dear student, your request has been updated on collpoll, kindly check
Dear Student, there was no error in the system, now changing of batch is not possible
Window is already closed, you can again try your luck in the next semester.
Dear Student,  You need to send the add/drop form mentioning the course you want to drop then it will be further taken to the approvals by me. You can find the add/drop form in the mail shared by Registry at the time of Registration process for Odd term 2
Dear Shubham, I have already replied to your query before, so I am closing this ticket.
Dear Tanish, kindly mail to c-saphal.anand@bennett.edu.in  or  Saphal Anand <saphal.a@collpoll.com> , for your collpoll queries to resolve
to get the list kindly contact to your faculty mentor
Dear Student,
you can get the form in the mail sent by Registrar before the registration in

In [4]:
lm = WordNetLemmatizer()
ourClasses = []
newWords = []
documentX = []
documentY = []
for intent in data.keys():
    for pattern in data[intent]['Patterns']:
        ourTokens = nltk.word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent)

    if intent not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent)

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [5]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:, 0]))
y = num.array(list(trainingData[:, 1]))

In [6]:
iShape = (len(x[0]),)
oShape = len(y[0])
# parameter definition
ourNewModel = Sequential()
# In the case of a simple stack of layers, a Sequential model is appropriate

# Dense function adds an output layer
ourNewModel.add(Dense(128, input_shape=iShape, activation="sigmoid"))
# The activation function in a neural network is in charge of converting the node's summed weighted input into activation of the node or output for the input in question
ourNewModel.add(Dropout(0.5))
ourNewModel.add(Dense(64, activation="sigmoid"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
md = tensorF.keras.optimizers.Adam(learning_rate=0.01,)
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
ourNewModel.fit(x, y, epochs=400, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               219776    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 27)                1755      
                                                                 
Total params: 229,787
Trainable params: 229,787
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/400
19/19 [==========================

In [7]:
l=[x for x in data.keys()]
l

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26']

In [8]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, data):
  brr=" "
  tag = firstlist[0]
  listOfIntents = [x for x in data.keys()]
  for i in listOfIntents:
    if i == tag:
      #print("waaw")
      brr = random.choice(data[i]['Responses'])
      break
  return brr

In [11]:
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newWords, ourClasses)
    print(getRes(intents, data))


1/1 [==============================] - 0s 20ms/step
Window is already closed, you can again try your luck in the next semester.
1/1 [==============================] - 0s 19ms/step
Dear Student,

You need to send the add/drop form mentioning the course you want to drop then it will be further taken to the approvals by me.
1/1 [==============================] - 0s 18ms/step
Dear Rajat, we have relative marking for your scores.
1/1 [==============================] - 0s 22ms/step
Work in progress.
1/1 [==============================] - 0s 24ms/step
Dear Student,

you can look into Archives on CSEBU Webapp only.
PFA syllabus for the Course CSET492.
1/1 [==============================] - 0s 19ms/step
Dear Student,

Kindly consult with Finance Department Regarding Fee related Query.
Thanks.
1/1 [==============================] - 0s 21ms/step
Today i have tried contact with you but your mobile no was switch off so kindly contact to me ASAP.


KeyboardInterrupt: Interrupted by user